## **1**

In [1]:
import pandas as pd

In [7]:
data = {'a':[0,0,0,0,0,0,0,0], "b": [0,2,0,1,0,0,0,0],
       "c":[0,0,2,0,0,1,1,0], "d": [0,2,0,1,1,3,0,0], "e": [0,1,2,3,1,4,0,0],
       "f":[0,3,2,1,1,1,5,0], "g": [0,2,2,1,0,0,2,0], "h": [0,0,0,0,0,0,0,0]}

frame = pd.DataFrame(data)
frame

,a,b,c,d,e,f,g,h
0,0,0,0,0,0,0,0,0
1,0,2,0,2,1,3,2,0
2,0,0,2,0,2,2,2,0
3,0,1,0,1,3,1,1,0
4,0,0,0,1,1,1,0,0
5,0,0,1,3,4,1,0,0
6,0,0,1,0,0,5,2,0
7,0,0,0,0,0,0,0,0


In [8]:
filtered_data = {'a':[4,0,1,1,1,0], "b": [0,5,1,4,1,1],
       "c":[4,3,4,5,3,1], "d": [3,5,4,3,10,3], "e": [5,4,3,4,4,9],
       "f":[2,5,3,1,1,3]}
frame2 = pd.DataFrame(filtered_data)
frame2

,a,b,c,d,e,f
0,4,0,4,3,5,2
1,0,5,3,5,4,5
2,1,1,4,4,3,3
3,1,4,5,3,4,1
4,1,1,3,10,4,1
5,0,1,1,3,9,3


In [10]:
maxpool_output = {'a':[5,4,1], "b": [5,5,10],
       "c":[5,4,9]}

frame3 = pd.DataFrame(maxpool_output)
frame3

,a,b,c
0,5,5,5
1,4,5,4
2,1,10,9


## **Basic Model for CNN**

In [15]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)
    

In [17]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [18]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform = transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform = transforms.ToTensor(),
                        download=True)

In [19]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True, drop_last=True)


In [26]:
class CNN(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out
    

In [27]:
model = CNN().to(device)

In [28]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [29]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')    

Learning started. It takes sometime.
[Epoch:    1] cost = 0.190865695
[Epoch:    2] cost = 0.0506182984
[Epoch:    3] cost = 0.0350851528
[Epoch:    4] cost = 0.0284913946
[Epoch:    5] cost = 0.0234592315
[Epoch:    6] cost = 0.0173421912
[Epoch:    7] cost = 0.0161922015
[Epoch:    8] cost = 0.0138808591
[Epoch:    9] cost = 0.0123546245
[Epoch:   10] cost = 0.0100175506
[Epoch:   11] cost = 0.0105740028
[Epoch:   12] cost = 0.0108399857
[Epoch:   13] cost = 0.00683387741
[Epoch:   14] cost = 0.0106311142
[Epoch:   15] cost = 0.00685142493
Learning Finished


In [30]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

C:\Users\DongHwi Kim\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\DongHwi Kim\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.9911999702453613


## **2-1**

In [37]:
class CNN2(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN2,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=4, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=2, padding=1),
                                         torch.nn.ReLU(), torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(2*2*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out
    

In [38]:
model = CNN2().to(device)

In [39]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [40]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')    

Learning started. It takes sometime.
[Epoch:    1] cost = 0.25038445
[Epoch:    2] cost = 0.0650188699
[Epoch:    3] cost = 0.0465202294
[Epoch:    4] cost = 0.0361797996
[Epoch:    5] cost = 0.0297054015
[Epoch:    6] cost = 0.0267538242
[Epoch:    7] cost = 0.0199700203
[Epoch:    8] cost = 0.0188587494
[Epoch:    9] cost = 0.0161151141
[Epoch:   10] cost = 0.0139145628
[Epoch:   11] cost = 0.0146521777
[Epoch:   12] cost = 0.0100352885
[Epoch:   13] cost = 0.0121229524
[Epoch:   14] cost = 0.0103363497
[Epoch:   15] cost = 0.0086538
Learning Finished


In [42]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9866999983787537


## **2-2**

### Layer3의 Output Size = (2 * 2 * 128)

## **2-3**

Filter가 커져서 좀 덜 세밀하게 값을 가져오고, stride 또한 커졌기에
Cost는 이전보다 증가하고, Accuracy 또한 이전보다 증가할 것 같다.

## **2-4**

Cost는 이전보다 증가하였고, Accuracy는 이전보다 감소하였다